In [8]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import pandas as pd
import pickle
import re
import csv
import random

Read in the pickle file and find the top five occurance rate payloads in the file
Return the payloads as 'text'讀取出text 

In [3]:
with open("pop_report_with_tknscore_new.pkl", "rb") as f:
    pop_df = pickle.load(f)

with open("imap_report_with_tknscore_new.pkl", "rb") as f:
    imap_df = pickle.load(f)

with open("smtp_report_with_tknscore_new.pkl", "rb") as f:
    smtp_df = pickle.load(f)

with open("sip_report_with_tknscore_new.pkl", "rb") as f:
    sip_df = pickle.load(f)

label_df = pd.read_csv("sid_table(packet).csv")


## 流程
### 獲取目標資料
- 獲取特定資料集的所有SID(unique)
- 獲取各個SID所對應到的text
- 過濾出sizt(text)>5的SID
- 對這些SID用LLM自動生成regex(生成Regex流程)
### 生成Regex
- 對每一個SID給LLM幾個regex例子
- 要求LLM生成三個不一樣但是可以過濾text的regex
### 測試Regex
- 拿答案給的regex和我們生成的regex測試能否的到相同結果(2/3過算對)

In [4]:
#獲取特定資料集的所有SID
def get_unique_sids_list(df):
    unique_sids = df[df['text'].notnull()]['sid'].unique()
    return unique_sids
sidList = get_unique_sids_list(pop_df)
print(sidList)
#print(len(sidList))

['1454621' '1335091' '1783777' '1648627' '1341424' '1213411' '1988637'
 '1407638' '1176066' '1161912' '1415573' '1291514' '3000002' '1872707'
 '1987844' '1680903' '1734662' '3000001' '1764655' '1848442' '1652949'
 '1215197' '1491662' '1419527' '1397090' '1414500' '1602037' '1972911'
 '1226089' '1547453' '1398061' '1604683' '1776829' '1016723' '1292430'
 '1018113' '1063876' '1396997' '1852737' '3000006']
40


In [5]:
#獲取各個SID所對應到的text
def map_sid_to_unique_texts(df):
    sid_to_texts = {}
    for _, row in df.iterrows():
        sid = row['sid']
        text = row['text']
        if pd.notnull(text):  # Ensure text is not NaN
            if sid in sid_to_texts:
                sid_to_texts[sid].add(text)
            else:
                sid_to_texts[sid] = {text}
    return sid_to_texts
sid_to_unique_texts = map_sid_to_unique_texts(pop_df)
#print(sid_to_unique_texts)

In [6]:
#過濾掉text長度小於5的sid
def filter_sids_by_text_length(sit_to_text_dict):
    filtered_sids = set()
    for key, value in sit_to_text_dict.items():
        if len(value) > 5:  # Convert to string to avoid errors with non-string types
            filtered_sids.add(key)
    return filtered_sids

# Assuming pop_df is your DataFrame
filtered_sids = filter_sids_by_text_length(sid_to_unique_texts)
#print(filtered_sids)
#print(len(filtered_sids))

In [9]:
def get_pcre_by_sid(sid):
    with open('sid_table(packet).csv', newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            if row['SID'] == str(sid):
                # Remove leading and trailing slashes from the pcre value
                return row['pcre'].strip('/')
    return None

# Iterate Throush Positive payload text to see if the ans regex is correct    
def is_ans_correct(sid):
    correct = 0
    total = 0
    errorList = []
    ansPattern = get_pcre_by_sid(str(sid))
    print(f"ansPattern: {ansPattern}")
    sid_to_unique_texts = map_sid_to_unique_texts(pop_df)
    texts = sid_to_unique_texts[sid]
    for text in texts:
        total = total + 1
        correct = correct + bool(re.search(ansPattern, text))
        if not match_patterns(text, threeAnsPattern, ansPattern):
            errorList.append(text)

    print("answer test")
    print(f"correct: {correct}, total: {total}")
    print(f"errorList: {errorList}")
    return correct, total
#is_ans_correct('1783777')

### ChatGPT API


In [ ]:
load_dotenv()
CHAT_GPT_API_KEY = os.getenv('C_API_KEY')
client = OpenAI(
    api_key=CHAT_GPT_API_KEY
)

completion = client.chat.completions.create(
  model="gpt-3.5-turbo-1106",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Hello!"}
  ]
)

print(completion.choices[0].message)


### Gemini API

In [ ]:
load_dotenv()

system_prompt = "You are a cat. Your name is Neko."
user_prompt = "What is your name?"
GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')

genai.configure(api_key=GEMINI_API_KEY)

model=genai.GenerativeModel(
  model_name="gemini-1.5-flash",
  system_instruction=system_prompt)

response = model.generate_content(user_prompt)
print(response.text)

### Evaluation Generated Regex
Randonly take 20 positive and 20 negateve data as input to generated regex and answer regex, comparing there result.

In [11]:

#隨機選擇除了給定的SID以外的100個text
def select_random_texts(sid_to_unique_texts, given_sid, num_texts=100):
    # Filter out the given SID
    filtered_texts = [texts for sid, texts in sid_to_unique_texts.items() if sid != given_sid]
    
    # Flatten the list of lists to a single list of texts
    all_texts = [text for sublist in filtered_texts for text in sublist]
    
    # Randomly select 100 texts, or all texts if there are fewer than 100
    selected_texts = random.sample(all_texts, min(len(all_texts), num_texts))
    
    return selected_texts

def get_pcre_by_sid(sid):
    with open('sid_table(packet).csv', newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            if row['SID'] == str(sid):
                # Remove leading and trailing slashes from the pcre value
                return row['pcre'].strip('/')
    return None

def match_patterns(targetText, GeneratedPatternList,isPositive=True):
    non_matching_patterns = []  # Step 1: Initialize list for non-matching patterns
    # Check each pattern and add non-matching ones to the list
    for i, pattern in enumerate(GeneratedPatternList):
        if not re.search(pattern, targetText):
            non_matching_patterns.append(f"Pattern {i+1}: {pattern}")
    ourResult = True if (9-len(non_matching_patterns)) >= 7 else False
    with open('evaluation_result.txt', 'w') as file:
        for pattern in non_matching_patterns:
            file.write(pattern)
    #ansMatch = bool(re.search(ansPattern, targetText))
    return ourResult == isPositive
def positive_evaluation(threeAnsPattern,sid):
    correct = 0
    total = 0
    errorList = []
    ansPattern = get_pcre_by_sid(str(sid))
    print(f"ansPattern: {ansPattern}")
    sid_to_unique_texts = map_sid_to_unique_texts(pop_df)
    texts = sid_to_unique_texts[sid]
    for text in texts:
        total = total + 1
        correct = correct + match_patterns(text, threeAnsPattern, True)
        if not match_patterns(text, threeAnsPattern, True):
            errorList.append(text)

    print("positive test")
    print(f"correct: {correct}, total: {total}")
    print(f"errorList: {errorList}")
    return correct, total

def negative_evaluation(GeneratedPatternList,sid):
    correct = 0
    total = 0
    errorList = []
    ansPattern = get_pcre_by_sid(str(sid))
    texts = select_random_texts(sid_to_unique_texts, sid)
    for text in texts:
        total = total + 1
        correct = correct + match_patterns(text, GeneratedPatternList, False)
        if  not match_patterns(text, GeneratedPatternList, False):
            errorList.append(text)
    print("negative test")
    print(f"correct: {correct}, total: {total}")
    for error_text in errorList:
        print(error_text)
    print(f"errorList: {errorList}")
    
regex1 = "GET .+ HTTP/1\.[01]\r\nHost: [^\r\n]+(\r\n[^\r\n]+)*\r\n\r\n"
regex2 = "^GET\s.+\sHTTP\/1\.[01]\r\nHost:\s[^\r\n]+(\r\nUser-Agent:\s[^\r\n]+)?(\r\nAccept:[^\r\n]+)?(\r\nAccept-Encoding:[^\r\n]+)?\r\n\r\n$"
regex3 = "^GET\s.+\sHTTP\/1\.[01]\r\nHost:\s[^\r\n]+(\r\n[^\r\n]+)*\r\n\r\n$"
regex4 = "^GET /.* HTTP/1\.[01]\r\nHost: [\d\w\.\-]+(:\d+)?\r\nUser-Agent: .+\r\nAccept: \*/\*\r\nAccept-Encoding: gzip\r\n\r\n$"
regex5 = "^GET /.* HTTP/1\.[01]\r\nHost: [\w\d\.\-]+(:\d+)?\r\n(User-Agent: .+\r\n)?Accept: \*/\*\r\n(Accept-Encoding: gzip\r\n)?\r\n$"
regex6 = "^GET /.* HTTP/1\.[01]\r\nHost: [\d\w\.\-]+(:\d+)?\r\nUser-Agent: Mozilla/5\.0.*\r\nAccept: \*/\*\r\nAccept-Encoding: gzip\r\n\r\n$"
regex7 = "^GET\s\S*\sHTTP/1\.[01]\r\n(Host: .+\r\n)?(User-Agent: .+\r\n)?(Accept: .+\r\n)?(Accept-Encoding: .+\r\n)?(\r\n)+$"
regex8 = "^GET\s\S*\sHTTP/1\.[01]\r\n(Host: .+\r\n)(User-Agent: .+\r\n)?(Accept: .+\r\n)?(Accept-Encoding: .+\r\n)?\r\n$"
regex9 = "^GET\s\S*\sHTTP/1\.[01]\r\n(Host: .+\r\n)?(User-Agent: .+\r\n)?(Accept: .*\r\n)?(Accept-Encoding: .*\r\n)?\r\n\r\n$"
GeneratedPatternList = [
    regex1, regex2, regex3, regex4, regex5, regex6, regex7, regex8, regex9
]

sid = '1783777'
positive_evaluation(GeneratedPatternList, sid)
negative_evaluation(GeneratedPatternList, sid)

ansPattern: ^(GET|HEAD|POST|OPTIONS)( )(\/|\/version|\/api.*|\/jsproxy)( )(HTTP\/1\.(?:0|1))(\r\n)(.*)$
positive test
correct: 584, total: 684
errorList: ['GET / HTTP/1.0\r\nHost: 61.67.24.45:110\r\nAccept: */*\r\n\r\n', 'GET / HTTP/1.1\r\nHost: 210.203.16.14:110\r\nUser-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10.11; rv:47.0) Gecko/20100101 Firefox/47.0\r\nAccept: */*\r\nConnection: keep-alive\r\n\r\n', 'GET / HTTP/1.0\r\nHost: 61.67.24.29:110\r\nAccept: */*\r\n\r\n', 'GET / HTTP/1.0\r\nHost: 222.156.247.14:110\r\nAccept: */*\r\n\r\n', 'GET / HTTP/1.0\r\nHost: 43.255.181.6:110\r\nAccept: */*\r\n\r\n', 'GET / HTTP/1.0\r\nHost: 203.187.87.60\r\n\r\n', 'GET / HTTP/1.0\r\nHost: 61.67.24.54:110\r\nAccept: */*\r\n\r\n', 'GET / HTTP/1.0\r\nHost: 61.67.24.9:110\r\nAccept: */*\r\n\r\n', 'GET / HTTP/1.0\r\nHost: 43.255.181.5:110\r\nAccept: */*\r\n\r\n', 'GET / HTTP/1.0\r\nHost: 210.242.148.2:110\r\nAccept: */*\r\n\r\n', 'GET / HTTP/1.1\r\nHost: 103.225.3.23:110\r\n\r\n', 'GET /version HTTP

### Helper Function

In [ ]:
def list_csv_columns():
    with open('sid_table(packet).csv', newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        # Print all column names
        print(reader.fieldnames)

In [ ]:
#print filtered_sids and corresponding text
for sid in filtered_sids:
    print(f"sid: {sid}, text: {sid_to_unique_texts[sid]}")

In [76]:

protocols = [
    {'name': 'pop_df', 'data': pop_df},
    {'name': 'imap_df', 'data': imap_df},
    {'name': 'smtp_df', 'data': smtp_df},
    {'name': 'sip_df', 'data': sip_df}
]

with open('target.txt', 'w') as file:
    pass

with open('target.txt', 'a') as file:
    for protocol in protocols:
        protocol_name = protocol['name']
        protocol_data = protocol['data']
        
        sid_to_unique_texts = map_sid_to_unique_texts(protocol_data)
        filtered_sids = filter_sids_by_text_length(sid_to_unique_texts)
        
        file.write(f"Protocol: {protocol_name}\n")
        
        for sid in filtered_sids:
            texts = list(sid_to_unique_texts[sid])
            random.shuffle(texts)
            selected_texts = texts[:50]
            file.write(f"sid: {sid}, text: {selected_texts}\n \n")
            file.write("Please find a regular expression to match all packet payloads.\n" + 
                       "You need to find the similarities in the sentences and generalize the parts where they differ. \n" + 
                        "The regular expression is in PCRE fromat, please be aware to evaluate the validity of the expression you generated under PCRE regulations. \n" + 
                        "There will be examples to help you find the patterns. \n"  +
                        "with the given example payloads: \n" +
                        "[‘DELE 3\\r\\n’, ‘DELE 128\\r\\n’, ‘DELE 74\\r\\n’, ‘DELE 22\\r\\n’, ‘DELE 70\\r\\n’] \n" +
                        "These examples show the attacker is trying to delete someone’s email by POP protocol. \n" +
                        "The index of the desired mail is indicated under the DELE command. \n" +
                        "Thus the best regular expression that matches them will be ‘^(DELE)( )(.*)(\\r\\n)$’ \n" + "\n" +
                        "With the given example payloads: \n" +
                        "[‘EHLO BtuCBHdSb51.com\\r\\n’, ‘EHLO 203.187.87.27\\r\\n’, ‘EHLO slae02Fo9Ep.com\\r\\n’, ‘EHLO 210.64.37.51\\r\\n’, ‘EHLO LLb0RwqdbkikFWo.com\\r\\n’] \n" + 
                        "These examples show the attacker is trying to make sure the SMTP server is up and running. " + 
                        "The command EHLO works in both lower case and uppercase, after that follows the SMTP server address. \n" +
                        "Thus the best regular expression to match them will be ‘^([E|e][H|h][L|l][O|o])(.*)(\\r\\\n)$’ \n" + "\n" +
                        "Next, with the given payloads: \n")
            file.write(f"{selected_texts}\n")         
            file.write("Please give 3 possible and different regular expressions to match all of the elements. \n" +
                        "You can give only 1 expression if the 3 expressions you find are too similar. \n" + 
                        "Let’s work this out in a step-by-step way to make sure we have the right answer. \n" + 
                        "To make the expression not too general, make sure the expressions don’t match these negative examples: [‘CAPA\\r\\n’, ‘CAPA\\r\\n’, ‘\\x15\\x03\\x01’, ‘GET / HTTP/1.0\\r\\n\\r\\n’, ‘r\\n\\r\\n’] \n")
        file.write("\n")

In [89]:
import random 

protocols = [
    {'name': 'pop_df', 'data': pop_df},
    {'name': 'imap_df', 'data': imap_df},
    {'name': 'smtp_df', 'data': smtp_df},
    {'name': 'sip_df', 'data': sip_df}
]

with open('target.txt', 'w') as file:
    pass

with open('target.txt', 'a') as file:
    for protocol in protocols:
        protocol_name = protocol['name']
        protocol_data = protocol['data']
        
        sid_to_unique_texts = map_sid_to_unique_texts(protocol_data)
        filtered_sids = filter_sids_by_text_length(sid_to_unique_texts)
        
        file.write(f"Protocol: {protocol_name}\n")
        
        for sid in filtered_sids:
            for i in range (3):
                texts = list(sid_to_unique_texts[sid])
                random.shuffle(texts)
                selected_texts = texts[:5]
                file.write(f"sid: {sid}, text: {selected_texts}\n")
        file.write("\n")

In [ ]:
with open('all_texts.txt', 'a') as file:
    for protocol in protocols:
        protocol_name = protocol['name']
        protocol_data = protocol['data']
        
        sid_to_unique_texts = map_sid_to_unique_texts(protocol_data)
        filtered_sids = filter_sids_by_text_length(sid_to_unique_texts)
        
        file.write(f"Protocol: {protocol_name}\n")
        
        for sid in filtered_sids:
            file.write(f"sid: {sid}, text: {sid_to_unique_texts[sid]}\n")

        file.write("\n")